In [ ]:
import pandas as pd
import sqlite3
import re
import urllib3
from transformers import pipeline

In [ ]:
col = []

def load_excel_to_sql(excel_file, db_file):
    df = pd.read_csv(excel_file)
    global col
    col = list(df.drop(columns=['Unnamed: 0']).columns)
    conn = sqlite3.connect(db_file)
    df.to_sql('vehicle', conn, if_exists='replace', index=False)
    print(f"Data successfully loaded into {db_file}")
    conn.close()

def extract_sql_query(response):
    try:
        match = re.search(r"(SELECT|UPDATE|DELETE|INSERT INTO|CREATE TABLE|ALTER TABLE).*?FROM.*?(;|$)", response, re.DOTALL | re.IGNORECASE)

        if match:
            sql_query = match.group(0).strip()
            if not sql_query.endswith(";"):
                sql_query += ";"
            return sql_query
        else:
            print(f"response: {response}")
            print("Failed to extract SQL query.")
            return None
    except Exception as e:
        print(f"Error in extraction logic: {e}")
        return None

def clean_sql_query(sql_query):
    sql_keywords = ["SELECT", "FROM", "WHERE", "UPDATE", "DELETE", "INSERT", "CREATE", "ALTER", "GROUP BY", "ORDER BY", "HAVING"]

    cleaned_query = "\n".join([line for line in sql_query.splitlines() if any(keyword in line.upper() for keyword in sql_keywords)])

    if not cleaned_query.endswith(";"):
        cleaned_query += ";"

    return cleaned_query

def system_prompt():
    return '''
    You are a SQL query generator which takes input from the user and uses that input to genrate a query to provide the user with desired results.

    The table on which you will be working on is called 'vehicle' and it has the following columns:
    id, region, price, year, manufacturer, model, fuel, odometer, transmission, size, type, state

    Example:
    input: provide list of id of vehicle which has price greater than 150000

    Expected output:
    SELECT id, price
    FROM vehicle
    where price > 150000;

    The above is just and example. Do not take it to be a fixed utput.

    Only generate the SQL query based on the input. Do not add conditions by your own. Do not provide any other output, only generate the SQL query.

    Input:
    '''

def generate_sql_query(user_input, sql_generator):
    prompt = system_prompt() + user_input

    try:
        response = sql_generator(prompt, max_length=len(prompt)+1, num_return_sequences=1, truncation=True, pad_token_id=50256)[0]['generated_text']

        sql_query = extract_sql_query(response)

        if sql_query:
            sql_query = clean_sql_query(sql_query)
            return sql_query
        else:
            print("Failed to extract SQL query.")
            return None
    except Exception as e:
        print(f"Error generating SQL query: {e}")
        return None

def execute_sql_query(db_file, sql_query):
    conn = sqlite3.connect(db_file, timeout=30)
    cursor = conn.cursor()

    try:
        cursor.execute(sql_query)
        results = cursor.fetchall()
    except sqlite3.Error as e:
        print(f"An error occurred: {e}")
        results = []

    cursor.close()
    conn.close()

    return results

def handle_user_input(user_input, db_file, sql_generator):
    sql_query = generate_sql_query(user_input, sql_generator)

    if sql_query:
        print(f"Generated SQL Query: {sql_query}")

        header = set()
        for cols in sql_query.split(" "):
            cols = cols.split(",")[0]
            if cols in col:
                header.add(cols)
        header = list(header)
        results = execute_sql_query(db_file, sql_query)
        if results:
            print("Query Results:")
            display(pd.DataFrame(data = results, columns=header))
        else:
            print("No results found or an error occurred.")
    else:
        print("Failed to generate SQL query.")

excel_file = "vehicle.csv"
db_file = "vehicle.db"
load_excel_to_sql(excel_file, db_file)

Data successfully loaded into vehicle.db


In [ ]:
sql_generator = pipeline('text-generation', model='EleutherAI/gpt-neo-2.7B')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/10.7G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [41]:
while True:
    user_input = input("Please enter your SQL question (type 'exit' to quit): ")
    if user_input.lower() == 'exit':
        print("Chat Ended")
        break
    handle_user_input(user_input, db_file, sql_generator)

Please enter your SQL question (type 'exit' to quit): select id of cars having price greather than 150000
Generated SQL Query: SELECT id, price FROM vehicle where price > 150000;
Query Results:


,price,id
0,7310125042,229500
1,7305265540,1111111111
2,7302895209,2000000


Please enter your SQL question (type 'exit' to quit): exit
Chat Ended
